# 專案：應用於臺北捷運時序資料之深度學習分析框架

## 筆記本 01：資料獲取與預處理

**目標：**
1.  從「臺北市資料大平臺」下載所有月份的 [捷運 OD 流量資料](https://data.taipei/dataset/detail?id=63f31c7e-7fc3-418b-bd82-b95158755b4d)。
2.  將所有分散的 CSV 檔案合併成一個單一的 DataFrame。
3.  對合併後的資料進行初步的檢視與清理。

In [1]:
import os
import sys

# 將 src 目錄添加到 Python 的模組搜索路徑中
# 這樣我們才能導入自己寫的 data_loader 模組
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"專案根目錄: {project_root}")
print(f"原始碼目錄已加入 sys.path: {src_path}")

專案根目錄: c:\Users\asd47\OneDrive - rui0828\master\高等人工智慧\final_project\MRT_Traffic_Analysis
原始碼目錄已加入 sys.path: c:\Users\asd47\OneDrive - rui0828\master\高等人工智慧\final_project\MRT_Traffic_Analysis\src


## 步驟一：下載原始資料

此步驟的目標是透過 `src/data_loader.py` 中的 `DataLoader` 類別，自動從臺北市資料大平臺抓取所有月份的 OD 流量 CSV 檔案，並儲存至 `data/raw/` 目錄下。

In [2]:
import os
import sys

# --- 再次確認環境設定 ---
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
    
from data_loader import DataLoader

# --- 設定正確的參數 ---
# 臺北捷運每日分時各站OD流量統計資料: https://data.taipei/dataset/detail?id=63f31c7e-7fc3-418b-bd82-b95158755b4d
DATASET_ID = 'eb481f58-1238-4cff-8caa-fa7bb20cb4f4' # API 的 ID 跟 網址上的 ID 會不一樣
RAW_DATA_DIR = os.path.join(project_root, 'data', 'raw')

# --- 執行下載 ---
print("=== 開始執行最終下載程序 ===")
loader = DataLoader(dataset_id=DATASET_ID, limit=1000)
loader.download_monthly_data(save_dir=RAW_DATA_DIR)

# --- 驗證下載結果 ---
print("\n--- 下載結果驗證 ---")
try:
    downloaded_files = os.listdir(RAW_DATA_DIR)
    if downloaded_files:
        print(f"在 '{RAW_DATA_DIR}' 目錄中找到 {len(downloaded_files)} 個檔案:")
        # 列出前5個和後5個檔案作為範例
        sorted_files = sorted(downloaded_files)
        print("頭5個檔案:", sorted_files[:5])
        print("末5個檔案:", sorted_files[-5:])
    else:
        print("目錄為空，請檢查下載過程是否有錯誤訊息。")
except FileNotFoundError:
    print(f"錯誤：找不到目錄 '{RAW_DATA_DIR}'。")

=== 開始執行最終下載程序 ===
正在從 API 獲取資料集資源列表: https://data.taipei/api/v1/dataset/eb481f58-1238-4cff-8caa-fa7bb20cb4f4?scope=resourceAquire&limit=1000
成功解析並提取到 100 個資源連結。

準備開始下載 100 個檔案至 'c:\Users\asd47\OneDrive - rui0828\master\高等人工智慧\final_project\MRT_Traffic_Analysis\data\raw'...


下載進度: 100%|██████████| 100/100 [16:51<00:00, 10.11s/it]


所有檔案下載完成。

--- 下載結果驗證 ---
在 'c:\Users\asd47\OneDrive - rui0828\master\高等人工智慧\final_project\MRT_Traffic_Analysis\data\raw' 目錄中找到 100 個檔案:
頭5個檔案: ['201701.csv', '201702.csv', '201703.csv', '201704.csv', '201705.csv']
末5個檔案: ['202412.csv', '202501.csv', '202502.csv', '202503.csv', '202504.csv']


## 步驟二：讀取與合併資料
將所有月份的原始 CSV 檔合併成單一 DataFrame，並進行初步檢視。

In [ ]:
from data_processor import load_and_merge_raw_data
import pandas as pd

# 設定 pandas 顯示選項，以便更好地觀察 DataFrame
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# --- 執行讀取與合併 ---
# RAW_DATA_DIR 變數我們在之前的儲存格已經定義過了
mrt_df = load_and_merge_raw_data(RAW_DATA_DIR)


# --- 初步檢視合併後的 DataFrame ---
if not mrt_df.empty:
    print("\n--- 合併後 DataFrame 的基本資訊 ---")
    mrt_df.info()
    
    print("\n\n--- 資料頭5筆 (Head) ---")
    display(mrt_df.head())
    
    print("\n\n--- 資料末5筆 (Tail) ---")
    display(mrt_df.tail())
    
    print(f"\n\n合併後的總筆數為: {len(mrt_df):,}")
else:
    print("DataFrame 是空的，請檢查前面的步驟。")

找到 100 個 CSV 檔案，準備讀取與合併...


正在讀取檔案:  33%|███▎      | 33/100 [01:38<02:43,  2.44s/it]

讀取檔案 '201909.csv' 時發生錯誤: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.


正在讀取檔案:  36%|███▌      | 36/100 [01:47<03:04,  2.88s/it]